In [ ]:
import pandas as pd
#Dict approach
import glob
import os
import sys
from pathlib import Path
#from typing import Any, Optional
import numpy as np
import pandas as pd
#import torch
#from torch.utils.data import Dataset, DataLoader
#import pytorch_lightning as pl
import h5py

In [ ]:
#Load genome loader for salk cluster
sys.path.append('/iblm/netapp/home/jjaureguy/genome_loader/genome-loader')

sample_path = os.listdir('/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq')
genos_path = os.listdir('/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/19_genos_fastq')
total_path = sample_path + genos_path
test_fasta= '/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/hg38/ref_genome/hg38.fa'


out_dir = '/iblm/netapp/data4/jjaureguy/out_dir'

In [ ]:
#Load genome loader for runai-GPU cluster
sys.path.append('/home/jovyan/home/jjaureguy/genome_loader/genome-loader')

sample_path = os.listdir('/home/jovyan/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq')
genos_path = os.listdir('/home/jovyan/data3/jjaureguy/PRJEB18997/10_genos/19_genos_fastq')
total_path = sample_path + genos_path
test_fasta= '/home/jovyan/data3/jjaureguy/PRJEB18997/10_genos/hg38/ref_genome/hg38.fa'

# change to /iblm/netapp/data4/jjaureguy/ref_genome/WholeGenomeFasta/genome.fa



out_dir = '/home/jovyan/data4/jjaureguy/out_dir'

In [ ]:
#sys.path.append('/home/jovyan/w')
import genome_loader.write_h5
import genome_loader.encode_data
import genome_loader.get_data
import genome_loader.get_encoded
import genome_loader.load_data
import genome_loader.load_h5
from genome_loader.write_h5 import write_encoded_genome


# Filter Pandas DF Text VCF files

In [ ]:
df_1 = pd.read_csv('/iblm/netapp/data3/jjaureguy/PRJEB18997/genotypes/ftp_imputed.txt',sep='\t')
df_2 = pd.read_csv('/iblm/netapp/data3/jjaureguy/PRJEB18997/genotypes/ftp_ped.txt',sep='\t')
df_3 = pd.read_csv('/iblm/netapp/data3/jjaureguy/PRJEB18997/genotypes/ftp_tbi.txt',sep='\t')

In [ ]:

df_1 = df_1[df_1['imputed_phased genotypes.vcf.gz'].str.endswith('.gz')]

In [ ]:

df_1.rename(columns = {'imputed_phased genotypes.vcf.gz':'vcf'}, inplace = True)
df_1.reset_index(drop=True)

In [ ]:
df_2 = df_2[df_2['ped'].str.endswith('.gz')]
df_2.rename(columns = {'ped':'vcf'}, inplace = True)
df_2.reset_index(drop=True)

In [ ]:
df_3 = df_3[df_3['imputed_phased genotypes.vcf.gz.tbi'].str.endswith('.gz')]
df_3.rename(columns = {'imputed_phased genotypes.vcf.gz.tbi':'vcf'}, inplace = True)
df_3.reset_index(drop=True)

In [ ]:
frames = [df_1, df_2, df_3]
result = pd.concat(frames)

In [ ]:
result.reset_index(drop=True)

In [ ]:
result.to_csv('/iblm/netapp/data3/jjaureguy/PRJEB18997/phased_vcf/vcf.txt', sep='\t', index=False, header=None)

In [ ]:
total_path

# Write ohe ref genome

In [ ]:
write_encoded_genome(test_fasta, out_dir, h5_name='genome_onehot.h5', encode_spec= "ACGT")

# File paths

In [ ]:
# out_dir = "/iblm/netapp/data4/jjaureguy/out_dir"
treatments_path = os.listdir('/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-eipl_1/out/treatments')

# Create Bedfile DF of first 5(10 ids) interesected 

In [ ]:

unified_bed_df = pd.read_csv('/home/jovyan/data3/jjaureguy/PRJEB18997/10_genos/text_files/unified_peak_set/unified_peak.txt',sep='\t', header=None, names=['chrom', 'start', 'end'])


In [ ]:
# Check chromosome naming in list
unique = unified_bed_df['chrom'].unique()
# generate a list of chromosomes 1-22 without X and Y and mitochondrial 
chr_list =  [ 'chr{}'.format(x) for x in list(range(1,23)) ]



In [ ]:
# Filter chromosomes to chrom 1-22 without X and Y(only looking at autosomes)
unified_bed_df = unified_bed_df[unified_bed_df['chrom'].isin(chr_list)]
unified_bed_df['chrom'].unique()


In [ ]:
unified_bed_df

In [ ]:
# Reset index after filtering
unified_bed_df = unified_bed_df.reset_index(drop=True)

In [ ]:
unified_bed_df

In [ ]:
# Remove chromosome 3 and 2 out of chrom list to be used downstream for generating train/valid/test dataframes
chr_list.remove('chr3')
chr_list.remove('chr2')

In [ ]:
#check to make sure we have correct list for training data frame
chr_list

In [ ]:
# Set train/test/valid 
train_chrom = chr_list
valid_chrom = ['chr2']
test_chrom = ['chr3']

# Filter DF by boolean values for train/valid/test dataframes

In [ ]:


unified_bed_df["train"] = (unified_bed_df['chrom'].isin(train_chrom))
unified_bed_df["valid"] = (unified_bed_df['chrom'].isin(valid_chrom ))
unified_bed_df["test"] = (unified_bed_df['chrom'].isin(test_chrom))



In [ ]:
unified_bed_df

In [ ]:
#Check to make sure boolean values are correct
unified_bed_df['valid'].value_counts()



In [ ]:
#Check to make sure boolean values are correct
unified_bed_df['test'].value_counts()

In [ ]:
#Check to make sure boolean values are correct

unified_bed_df['train'].value_counts()

In [ ]:
# Set value variable to true
value = True
# Filter through boolean values and generate dataframes for train/test/valid
train_df = unified_bed_df.loc[unified_bed_df['train'] == True, ['chrom', 'start', 'end']]
valid_df = unified_bed_df.loc[unified_bed_df['valid'] == True, ['chrom', 'start', 'end']]
test_df = unified_bed_df.loc[unified_bed_df['test'] == True, ['chrom', 'start', 'end']]

#Generate csv text files of dataframes for dataset class



In [ ]:
test_df

In [ ]:
train_df = train_df.reset_index(drop=True)

In [ ]:
test_df = test_df.reset_index(drop=True)

In [ ]:
valid_df = valid_df.reset_index(drop=True)

In [ ]:
# generate CSV files of train/test/split dataframes for salk cluster
train_df.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/train_df.txt', sep='\t', index=False, header=True)
test_df.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/test_df.txt', sep='\t', index=False, header=True)
valid_df.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/valid_df.txt', sep='\t', index=False, header=True)


In [ ]:
# generate CSV files of train/test/split dataframes for runAI
train_df.to_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/train_df_runai.txt', sep='\t', index=False, header=True)
test_df.to_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/test_df_runai.txt', sep='\t', index=False, header=True)
valid_df.to_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/valid_df_runai.txt', sep='\t', index=False, header=True)


In [ ]:
unified_bed_df.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/unified_bed_df.txt', sep='\t', index=False, header=True)


In [ ]:
# Load back in already finished above.

unified_bed_df = pd.read_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/unified_bed_df.txt',sep='\t')


In [ ]:
unified_bed_df

In [ ]:
unified_bed_df['chrom'].unique()


# Create dataframe for patient ID, treatment type, H5 Read depth(atac output for tn5 fragment insertation cut sites), replace old files once completed


In [ ]:
column_names = ["patient_id","diff_frag_h5_path","int_gamma_diff_frag_h5_path","salm_int_gamma_diff_frag_h5_path","salm_diff_frag_h5_path"]

df_read_depth_h5 = pd.DataFrame(columns = column_names)

In [ ]:
df_read_depth_h5

In [ ]:
import glob
bam_paths = []
rest_bam_paths = []
peak_paths = []
treatment = []

for i in (total_path):
    #runai
    path = f'/home/jovyan/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/{i}/out/treatments/*/frag_depths.h5'
    path_rest = f'/home/jovyan/data3/jjaureguy/PRJEB18997/10_genos/19_genos_fastq/{i}/out/treatments/*/frag_depths.h5'
   
    # #salk
    # path = f'/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/{i}/out/treatments/*/frag_depths.h5'
    # path_rest = f'/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/19_genos_fastq/{i}/out/treatments/*/frag_depths.h5'
    for filename in glob.glob(path):
        bam_paths.append(filename)
    for filename in glob.glob(path_rest):
        rest_bam_paths.append(filename)

read_depth_frag_h5_paths = bam_paths + rest_bam_paths



In [ ]:
def write_h5_lens(in_h5, dataset_id=None):
    
    with h5py.File(in_h5, "a") as file:
        
        if not dataset_id:
            dataset_id = file.attrs["id"]

        for chrom in file.keys():
            chrom_data = file[chrom][dataset_id]
            file[chrom].attrs["length"] = chrom_data.shape[0]
            print(f"{chrom}: {file[chrom].attrs['length']}")

In [ ]:
for i in read_depth_frag_h5_paths:
    print('started writing h5 lens',{i})
    write_h5_lens(i)
    print('finished writing h5 lens',{i})
    


# Original code

In [ ]:
import re

id_re= re.compile(".*HPSI")
treatment_re = re.compile(".*_mb")
id_list = []
treatment_list = []
for i in read_depth_frag_h5_paths:
    p = Path(i)
    #print(p)
    split = (p.parts)
    id = list(filter(id_re.match, split))
    treatment = list(filter(treatment_re.match, split))
    id_list.append(id)
    treatment_list.append(id_list)
    

id_list = np.unique(id_list)
#df_read_depth_h5['patient_id'] = np.unique(id_list)
#df_read_depth_h5['treatment'] = treatment_list

In [ ]:
id_list

In [ ]:
# List of patient ids only containing 2 treatments
imbalanced_treatments = ['HPSI0913i-eika_2', 'HPSI0114i-iisa_3','HPSI1213i-nusw_2', 'HPSI1213i-pahc_4', 'HPSI0114i-joxm_1', 'HPSI1113i-ieki_2']

In [ ]:
 id_list = [x for x in id_list if x not in imbalanced_treatments]


In [ ]:
(id_list)

In [ ]:
df_read_depth_h5

In [ ]:
total_bam_paths = bam_paths + rest_bam_paths 
len(total_bam_paths)


In [ ]:
#id_list = [s for s in total_bam_paths if str(id_list) in s]

#new_list = [s for s in total_bam_paths if id_list in s]
# Using list comprehension
output = [b for b in total_bam_paths if
          all(a not in b for a in id_list)]
output

In [ ]:
total_bam_paths = [x for x in total_bam_paths if x not in output]


In [ ]:
len(total_bam_paths)

In [ ]:
total_bam_paths

In [ ]:
# All of the h5 file paths list
print(len(total_bam_paths))

# strings to be checked by file paths
diff = 'diff_mb'
int_gamma_diff = 'int_gamma_mb'
salm_int_gamma = 'salm_int_gamma_mb'
salm_diff = 'salm_mb'

# create sublist for colum and to be removed from total bam paths since total bam paths has 2 simiar regex pattern conflict
salm_int_gamma = [s for s in total_bam_paths if salm_int_gamma in s]
print('len of sam int gamma', len(salm_int_gamma))
# Create 
total_bam_paths = [x for x in total_bam_paths if x not in salm_int_gamma]

print('total bp',len(total_bam_paths))


#print((total_bam_paths))
#print(type(diff_mb))
# List comprehension to add paths to approriate treatment columns
diff = [s for s in total_bam_paths if diff in s]
int_gamma_diff = [s for s in total_bam_paths if int_gamma_diff in s]
salm_diff = [s for s in total_bam_paths if salm_diff in s]


# Set columns to new filtered lists of treatment paths
df_read_depth_h5['diff_frag_h5_path'] = diff
df_read_depth_h5['int_gamma_diff_frag_h5_path'] = int_gamma_diff
df_read_depth_h5['salm_int_gamma_diff_frag_h5_path'] = salm_int_gamma
df_read_depth_h5['salm_diff_frag_h5_path'] = salm_diff
#df_read_depth_h5['read_depth_frag_h5_path'] = total_bam_paths
#df_read_depth_h5['treatment'] = df_read_depth_h5['read_depth_frag_h5_path'].astype(str)

In [ ]:
df_read_depth_h5

In [ ]:
id_list

In [ ]:
import re

id_re= re.compile(".*HPSI")
treatment_re = re.compile(".*_mb")
id_list = []
print(id_list)
treatment_list = []
for i in df_read_depth_h5['diff_frag_h5_path']:
    p = Path(i)
    #print(p)
    split = (p.parts)
    id = list(filter(id_re.match, split))
    treatment = list(filter(treatment_re.match, split))
    id_list.append(id)
    treatment_list.append(id_list)
    

#id_list = np.unique(id_list)
df_read_depth_h5['patient_id'] = (id_list)
#df_read_depth_h5['treatment'] = treatment_list

In [ ]:
df_read_depth_h5

In [ ]:

df_read_depth_h5['patient_id'] = df_read_depth_h5['patient_id'].str[0]
#df_read_depth_h5['treatment'] = df_read_depth_h5['treatment'].str[0]



In [ ]:
df_read_depth_h5

In [ ]:
df_read_depth_h5_final = df_read_depth_h5.loc[df_read_depth_h5['imbalanced_treatments'] == False]


In [ ]:
df_read_depth_h5

In [ ]:
(df_read_depth_h5['patient_id'].value_counts())

In [ ]:
id_patient_list = df_read_depth_h5['patient_id'].unique()


In [ ]:
id_patient_list = np.array(id_patient_list.tolist())

In [ ]:
type(id_patient_list)
#23
#3
#3

In [ ]:
len(id_patient_list)

In [ ]:
import random
np.random.seed(0)
train_list = np.random.choice(id_patient_list, 19, replace=False)

In [ ]:
train_list


In [ ]:
remainder_list = set(id_patient_list)-set(train_list)
remainder_list = list(remainder_list)
remainder_list

In [ ]:
np.random.seed(0)
valid_list = np.random.choice(remainder_list, 2, replace=False)

In [ ]:
valid_list

In [ ]:
test_list = set(remainder_list)-set(valid_list)
test_list = list(test_list)



In [ ]:
test_list

In [ ]:
df_read_depth_h5["train"] = (df_read_depth_h5['patient_id'].isin(train_list))
df_read_depth_h5["valid"] = (df_read_depth_h5['patient_id'].isin(valid_list))
df_read_depth_h5["test"] = (df_read_depth_h5['patient_id'].isin(test_list))


In [ ]:
df_read_depth_h5['train'].value_counts()

In [ ]:
df_read_depth_h5['valid'].value_counts()

In [ ]:
df_read_depth_h5['test'].value_counts()

In [ ]:
df_read_depth_h5

In [ ]:
# Set value variable to true
value = True
# Filter through boolean values and generate dataframes for train/test/valid
train_df_tn5 = df_read_depth_h5.loc[df_read_depth_h5['train'] == True, ['patient_id', 'diff_frag_h5_path', 'int_gamma_diff_frag_h5_path','salm_int_gamma_diff_frag_h5_path', 'salm_diff_frag_h5_path']]
valid_df_tn5 = df_read_depth_h5.loc[df_read_depth_h5['valid'] == True, ['patient_id', 'diff_frag_h5_path', 'int_gamma_diff_frag_h5_path', 'salm_int_gamma_diff_frag_h5_path', 'salm_diff_frag_h5_path']]
test_df_tn5 = df_read_depth_h5.loc[df_read_depth_h5['test'] == True, ['patient_id', 'diff_frag_h5_path', 'int_gamma_diff_frag_h5_path', 'salm_int_gamma_diff_frag_h5_path', 'salm_diff_frag_h5_path']]

#Generate csv text files of dataframes for dataset class



In [ ]:

train_df_tn5 = train_df_tn5.reset_index(drop=True)
valid_df_tn5 = valid_df_tn5.reset_index(drop=True)
test_df_tn5 = test_df_tn5.reset_index(drop=True)

In [ ]:
train_df_tn5

In [ ]:
test_df_tn5

In [ ]:
valid_df_tn5

# Write CSVs for Train/valid/test and larger total df for Tn5 insertion cound dataframes for salk cluster

In [ ]:
#df_read_depth_h5.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/frag_tn5_h5_df.txt', sep='\t', index=False)
train_df_tn5.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/salk_df/train_frag_tn5_h5_df.txt', sep='\t', index=False)
valid_df_tn5.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/salk_df/valid_frag_tn5_h5_df.txt', sep='\t', index=False)
test_df_tn5.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/data_frames/salk_df/test_frag_tn5_h5_df.txt', sep='\t', index=False)

# Write CSVs for Train/valid/test and larger total df for Tn5 insertion cound dataframes for runAI cluster

In [ ]:
train_df_tn5.to_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/runai_df/train_frag_tn5_h5_df_runai.txt', sep='\t', index=False)
valid_df_tn5.to_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/runai_df/valid_frag_tn5_h5_df_runai.txt', sep='\t', index=False)
test_df_tn5.to_csv('/home/jovyan/data4/jjaureguy/jupyter/data_frames/runai_df/test_frag_tn5_h5_df_runai.txt', sep='\t', index=False)

# Create dictionary of bed and bam paths for 10 genos ids

In [ ]:
import glob
bam_paths = []
peak_paths = []
treatment_paths = []



column_names = [
    #'patient id', 
                'treatments','bam paths', 'peak paths']

d = {}

#filenames = [f for f in filenames if (f.startswith("ERR") and f.lower().endswith("1"))]
for i in (sample_path):
    #print([i][0]
    path = f"/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/{i}/out/treatments/*/merged_qc.bam"
    peak_path = f"/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/{i}/out/treatments/merged.bed"
    for filename in glob.glob(path):
        bam_paths.append(filename)
    for filename in glob.glob(peak_path):
        peak_paths.append(filename)
    for filename in (treatments_path):
        treatment_paths.append(filename)
    d[i] = pd.DataFrame(bam_paths)
    d[i].columns = ['bam paths']
    #d[i]['patient id'] = i
    d[i]['peak paths'] = pd.Series(peak_paths)
    d[i]['peak paths']= d[i]['peak paths'].fillna(method='ffill')
    d[i]['treatments']= pd.Series(treatment_paths)
    d[i]['treatments'] = d[i]['treatments'].str.replace('_mb', '')
    d[i] = d[i].reindex(columns=column_names)
    bam_paths.clear()
    peak_paths.clear()
    treatment_paths.clear()


# Create dictionary of bed and bam paths for 19 genos ids

In [ ]:
sample_path = os.listdir('/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/19_genos_fastq')


In [ ]:
import glob
bam_paths = []
peak_paths = []
treatment_paths = []



column_names = [
    #'patient id', 
                'treatments','bam paths', 'peak paths']

d_add = {}

#filenames = [f for f in filenames if (f.startswith("ERR") and f.lower().endswith("1"))]
for i in (sample_path):
    #print([i][0]
    path = f"/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/19_genos_fastq/{i}/out/treatments/*/merged_qc.bam"
    peak_path = f"/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/19_genos_fastq/{i}/out/treatments/merged.bed"
    for filename in glob.glob(path):
        bam_paths.append(filename)
    for filename in glob.glob(peak_path):
        peak_paths.append(filename)
    for filename in (treatments_path):
        treatment_paths.append(filename)
    d_add[i] = pd.DataFrame(bam_paths)
    d_add[i].columns = ['bam paths']
    #d[i]['patient id'] = i
    d_add[i]['peak paths'] = pd.Series(peak_paths)
    d_add[i]['peak paths']= d_add[i]['peak paths'].fillna(method='ffill')
    d_add[i]['treatments']= pd.Series(treatment_paths)
    d_add[i]['treatments'] = d_add[i]['treatments'].str.replace('_mb', '')
    d_add[i] = d_add[i].reindex(columns=column_names)
    bam_paths.clear()
    peak_paths.clear()
    treatment_paths.clear()

In [ ]:
d_add[i]

In [ ]:
df = pd.concat(d).reset_index().drop(columns='level_1').rename(columns={'level_0': 'patient_id'})
df_19 = pd.concat(d_add).reset_index().drop(columns='level_1').rename(columns={'level_0': 'patient_id'})

In [ ]:
df_19

In [ ]:
frames = [df, df_19]

result = pd.concat(frames)

In [ ]:
result

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
cur_offset = 0
bed_dfs = {}
for (patient_id, bam_path, peak_file), pdf in tqdm(result.groupby(['patient_id','bam paths', 'peak paths'])):
    
    bed_df = pd.read_csv(peak_file, delimiter='\t', 
                         header=None, names=['chrom', 'start', 'end'], 
                         dtype={'chrom':'category', 'start':int, 'end': int})
    
    bed_df['bam paths'] = bam_path
    bed_df['peak paths'] = peak_path
    bed_dfs[patient_id] = bed_df
dataset_df = pd.concat(bed_dfs, names=['patient_id']).reset_index()

In [ ]:
dataset_df

In [ ]:
del dataset_df['level_1']

In [ ]:
car = dataset_df['bam paths'].unique()

In [ ]:
car

In [ ]:
x

# Missing bam files

In [ ]:

x = result['bam paths'].unique()

c = np.setxor1d(x, car)
c



# Genome write read depth from bam files of df

# Generated partial list

In [ ]:
for fn in tqdm(dataset_df['bam paths'].unique()):
    fnp = Path(fn)
    outdir = fnp.parent
    write_read_depth(str(fnp), str(outdir))

# Rest of bam files missing from previous code

In [ ]:
for fn in tqdm(c):
    fnp = Path(fn)
    outdir = fnp.parent
    write_read_depth(str(fnp), str(outdir))

# Dataframe to text

In [ ]:
dataset_df.to_csv('/iblm/netapp/data4/jjaureguy/jupyter/atac_ml_project/data_frame/df.txt', sep='\t', index=False)